# TEDscraper Notebook

In [1]:
import pandas as pd
import re

import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import time
import random

import pickle

## Soup Maker

In [2]:
class SoupMaker:
    """Make soup objects and put your machine to sleep."""
    

    def sleep_short(self):
        """Suspends execution time between 0 - .2 seconds."""
        return time.sleep(random.uniform(0, .2))

    def sleep_long(self):
        """Suspends execution time between .5 - 2 seconds."""
        return time.sleep(random.uniform(.5, 2))

    def make_soup(self, url):
        """Returns soup object from a URL."""
        # generate random user-agent
        user_agent = {'User-agent': UserAgent().random}
        # request page and make soup
        page = requests.get(url, headers=user_agent)
        soup = BeautifulSoup(page.content, 'lxml')
        return soup


## CreateCSV

In [3]:
class CreateCSV(SoupMaker):
    """Create CSVs of TED topics and languages."""


    def create_topics_csv(self):
        """Creates CSV of all topics available from TED."""
        soup = self.make_soup('https://www.ted.com/topics')
        topic_list = []
        topic_tag = soup.find_all(class_='d:b', style='line-height:3;')
        for tag in topic_tag:
            topic = re.sub(r'\s+', '', tag.text)
            topic_list.append(topic)
        topics_series = pd.Series(topic_list, name='Topic')
        topics_series.to_csv('../data/topics.csv', index=False)

    def create_languages_csv(self):
        """Creates CSV of all language codes supported by TED."""
        lang_url = 'https://www.ted.com/participate/translate/our-languages'
        soup = self.make_soup(lang_url)
        lang_list = []
        lang_tags = soup.find_all('div', class_='h9')
        for tag in lang_tags:
            if tag.a == None:
                continue
            else:
                lang_code = re.search(r'(?<=\=)[\w-]+', tag.a['href']).group(0)
                lang_name = tag.text
                lang_list.append([lang_code] + [lang_name])
        lang_df = pd.DataFrame(data=lang_list, columns=['lang_code', 'language'])
        lang_df.to_csv('../data/languages.csv', index=False)


## Talk Features

In [4]:
class TalkFeatures(SoupMaker):
    """Class to get TED talk features."""


    def get_talk_id(self, soup):
        """Returns the talk_id provided by TED."""
        talk_id = re.search(r"(?<=\"current_talk\":)\"(\d+)\"", soup.text).group(1)
        return talk_id

    def get_title(self, soup):
        """Returns the title of the talk."""
        title_tag = soup.find(attrs={'name': 'title'}).attrs['content']
        title = title_tag.split(':')[1].strip()
        return title

    def get_speakers(self, soup):
        """Returns dict of all speakers per talk."""
        speaker_tag = re.findall(r"(?<=\"speakers\":).*?\"}]", soup.text)[0]
        # convert to DataFrame
        speakers_df = pd.read_json(speaker_tag)
        full_name_raw = (speakers_df.loc[:, 'firstname'] + ' '
                     + speakers_df.loc[:, 'middleinitial'] + ' '
                     + speakers_df.loc[:, 'lastname'])
        full_name_clean = full_name_raw.str.replace('\s+', ' ')
        # transform series to a dict
        speakers = full_name_clean.to_dict()
        return speakers

    def get_occupations(self, soup):
        """Returns list of the occupation(s) of the speaker(s) per talk."""
        occupations_tag = re.findall(r"(?<=\"speakers\":).*?\"}]", soup.text)[0]
        # convert json to DataFrame
        occupations_series = pd.read_json(occupations_tag)['description']
        if occupations_series.all():
            # clean and create dict
            occupations = occupations_series.str.lower().str.split(', ')
            occupations = occupations.to_dict()
        else:
            occupations = None
        return occupations

    def get_about_speakers(self, soup):
        """Returns dict with each 'About the Speaker' blurb per talk."""
        speaker_tag = re.findall(r"(?<=\"speakers\":).*?\"}]", soup.text)[0]
        # convert to DataFrame
        about_series = pd.read_json(speaker_tag)['whotheyare']
        if about_series.all():
            # transform series to a dict
            about_speakers = about_series.to_dict()
        else:
            about_speakers = None
        return about_speakers

    def get_views(self, soup):
        """Returns viewed count per talk."""
        view_count = re.search(r"(?<=\"viewed_count\":)\d+", soup.text).group(0)
        return view_count

    def get_recorded_date(self, soup):
        """Returns date a talk was recorded."""
        recorded_at = re.search(r"(?<=\"recorded_at\":)\"(.*?)T", soup.text).group(1)
        return recorded_at

    def get_published_date(self, soup):
        """Returns date a talk was published in TED.com."""
        published_at = soup.find(attrs={'itemprop': 'uploadDate'}).attrs['content']
        return published_at

    def get_event(self, soup):
        """Returns name of the event in which the talk was given."""
        event = re.search(r"(?<=\"event\":)\"(.*?)\"", soup.text).group(1)
        return event
    
    def get_native_lang(self, soup):
        """Returns native language code for each talk as a string."""
        native_lang = re.search(r'(?<=nativeLanguage\":\")[\w-]+', soup.text).group(0)
        return native_lang
    
    def get_available_lang(self, soup):
        """Returns list of all available languages (lang codes) for a talk."""
        languages = re.findall(r'(?<=languageCode\":\")[\w-]+', soup.text)
        clean_lang = sorted(list(set(languages)))
        return clean_lang

    def get_comments_count(self, soup):
        """Return the count of comments per talk."""
        try:
            comments_count = re.search(r"(?<=\"count\":)(\d+)", soup.text).group(1)
        except AttributeError:
            comments_count = None
        return comments_count

    def get_duration(self, soup):
        """Returns duration of a talk (format ex: 12M43S)"""
        duration_tag = soup.find(attrs={'itemprop': 'duration'}).attrs['content']
        duration = duration_tag.split('PT')[1]
        return duration

    def get_duration_sec(self, soup):
        """Returns duration of a talk in seconds."""
        duration =  re.search(r"(?<=\"duration\":)(\d+)", soup.text).group(1)
        return duration

    def get_topic_tags(self, soup):
        """Returns list of tags (topics) per talk."""
        match_obj = re.search(r"\"tag\":\"(.*?)\"", soup.text)
        tags = match_obj.group(1).split(',')
        return tags

    def get_related_talks(self, soup):
        """Returns dict (keys: id & title) of related talks."""
        related_tag = re.search(r"(?<=\"related_talks\":).*?]", soup.text).group(0)
        related_series = pd.read_json(related_tag)
        related_talks = related_series.loc[:, ['id', 'title']].to_dict()
        return related_talks

    def get_talk_url(self, soup):
        """Returns url for each talk as a string."""
        talk_tag = soup.find(attrs={'property': 'og:url'}).attrs['content']
        talk_url = talk_tag.split('/transcript')[0]
        return talk_url

    def get_talk_description(self, soup):
        """Returns description of the talk."""
        desc_tag = soup.find(attrs={'property': 'og:description'}).attrs['content']
        talk_desc = desc_tag.split(': ', 1)[1]
        return talk_desc

    def get_transcript(self, soup):
        """Returns talk's transcript as a single string.""" 
        transcript = ''
        transcript_strings = []
        for div in soup.find_all('div', class_="Grid__cell flx-s:1 p-r:4"):
            for p in div.find_all('p'):
                # add every string in the transcript to a list
                transcript_strings.append(" ".join(p.text.split()))
            else:
                # after all strings have been added, create a single transcript string
                transcript = " ".join(transcript_strings)
        return transcript


## URLs

In [50]:
class URLs(SoupMaker):
    """Get and process urls to scrape."""

    
    def topics_url_param(self):
        """Returns string of the url query from topics parameter."""
        topics_param = ''
        if self.topics != 'all':
            if isinstance(self.topics, list):
                for topic in self.topics:
                    topics_param += ('&topics[]=' + topic)
            else:
                raise ValueError("'topics' param needs to be a list")
        return topics_param

    def get_max_page(self):
        """Returns max pagination number from www.ted.com/talks."""
        page_num = [1]
        # make soup from ted.com/talks with specified language
        soup = self.make_soup(self.base_url + '&page=1&sort=newest')
        # iterate through each pagination element and get the max
        page_elem = soup.find_all('a', class_='pagination__item pagination__link')
        for element in page_elem:
            page_num.append(int(element.text))
        return max(page_num)
    
    def get_all_url_paths(self):
        """Returns list of all the talk url paths available in www.ted.com/talks"""
        url_path_list = []
        # construct url with lang code specified by the user
        talks_url = (self.base_url + '&page=')
        # set range from 1 to the max page in the pagination element
        page_range = range(1, self.get_max_page()+1)
        # iterate through each page and get the url for each talk
        for i in page_range:
            # try a second attempt if first attempt fails
            for attempt in range(2):
                try:
                    talks_page_url = talks_url + str(i) + '&sort=newest'
                    soup = self.make_soup(talks_page_url)
                    # delay between searches
                    self.sleep_short()
                    for div in soup.find_all('div', attrs={'class': 'media__image'}):
                        for a in div.find_all('a'):
                            url_path_list.append(a.get('href'))
                except:
                    # delay before continuing to second attempt
                    self.sleep_long()
                # break from attempts loop if no exceptions are raised
                else:
                    break
        return url_path_list

    def get_all_urls(self):
        """Returns list of complete urls for each talk's transcript page."""
        # '/talks/jen_gunter_why_can_t_we_talk_about_periods?language=fa'
        url_list = []
        for url in self.get_all_url_paths():
            url_list.append(('https://www.ted.com'
                             + url.replace(
                                 # to replace
                                 '?language=' + self.lang_code,
                                 # replace with
                                 '/transcript' + '?language=' + self.lang_code)
                            ))
        return url_list
    
    def clean_urls(self, urls):
        """Returns list of clean urls from urls the user inputs."""
        clean_urls = []
        for idx, url in enumerate(urls):
            if url.startswith('https://www.ted.com/talks'):
                parts = url.split('/')
                joined = '/'.join(parts[:5])
                clean = joined.split('?')
                lang = clean[0] + '/transcript?language=' + self.lang_code
                topic = lang + self.topics_url_param()
                clean_urls.append(lang)
            else:
                print(f'bad url @ {idx} >> {url}')
                continue
        return clean_urls
    
    def url_issues(self):
        """Returns DataFrame of urls with known issues."""
        issues_df = pd.read_csv('../data/urls_issues.csv')
        return issues_df
    
    def remove_urls_with_issues(self):
        """Remove urls with known issues to prevent unnecessary scraping."""
        urls = self.url_attribute()
        final_urls = []
        removed_urls = []
        removed_counter = 0
        issues_df = pd.read_csv('../data/urls_issues.csv')
        for url in urls:
            try:
                base_url = url.replace('transcript?language=' + self.lang_code, '')
                # is base url in the issues df?
                url_in_issues = (issues_df['url'] == base_url).any()
                # get the lang_codes of the base_url
                langs = issues_df.loc[issues_df['url'] == base_url, 'lang_code']
                # check if the url in issues_df
                if not url_in_issues:
                    final_urls.append(url)
                # if the url is in issues_df, check if it's for the same lang_code
                elif self.lang_code in langs.any():
                    removed_urls.append(url)
                    removed_counter += 1
                    continue
                else:
                    final_urls.append(url)
            except:
                removed_urls.append(url)
                removed_counter += 1
                continue
        if removed_urls:
            print(f"Removed the following {removed_counter} urls as they have "
                  "known issues:\n", removed_urls, end='\n\n')
        return final_urls


## TEDscraper

In [51]:
class TEDscraper(TalkFeatures, URLs):
    """Gets urls and scrapes TED talk data in the specified language.

    Attributes:
        lang_code (str): Language code. Defaults to 'en'.
        language (str): Language name derived from lang_code.
        urls (list): URLs of talks. Defaults to 'all'.
        topics (list): Talk topics. Defaults to 'all'.
        exclude (bool): Exclude transcript. Defaults to False.
        ted_dict (dict): Dict to store ted talk features after scraping.
        dict_id (int): Index of nested dict in 'ted_dict'.
        failed_counter: Counts urls that failed to get scraped.
    """
    

    def __init__(self, lang_code='en', urls='all', topics='all', exclude_transcript=False):
        self.lang_code = lang_code
        self.language = self.convert_lang_code()
        self.urls = urls
        self.topics = topics
        self.exclude = exclude_transcript
        self.ted_dict = {}
        self.dict_id = 0
        self.failed_counter = 0
        self.base_url = ('https://www.ted.com/talks'
                         + '?language=' + self.lang_code
                         + self.topics_url_param())
        
    def url_attribute(self):
        """Define urls attribute based on parameter 'urls'."""
        # define url attribute
        if self.urls == 'all':
            urls = self.get_all_urls()
        else:
            if isinstance(self.urls, list):
                urls = self.clean_urls(self.urls)
            else:
                raise ValueError("'urls' param needs to be a list")
        return urls

    def convert_lang_code(self):
        """Reads languages.csv and returns language.
        Parameters:
            lang_code (str): Language code
        """
        df = pd.read_csv('../data/languages.csv')
        lang_series = df.loc[(df['lang_code'] == self.lang_code), 'language']
        language = lang_series.values[0]
        return language
    
    def scrape_all_features(self, soup):
        """Scrapes all features to a nested dict."""
        # create nested dict
        self.ted_dict[self.dict_id] = {}
        nested_dict = self.ted_dict[self.dict_id]
        # add the features to the nested dict
        nested_dict['talk_id'] = self.get_talk_id(soup)
        nested_dict['title'] = self.get_title(soup)
        nested_dict['speakers'] = self.get_speakers(soup)
        nested_dict['occupations'] = self.get_occupations(soup)
        nested_dict['about_speakers'] = self.get_about_speakers(soup)
        nested_dict['views'] = self.get_views(soup)
        nested_dict['recorded_date'] = self.get_recorded_date(soup)
        nested_dict['published_date'] = self.get_published_date(soup)
        nested_dict['event'] = self.get_event(soup)
        nested_dict['native_lang'] = self.get_native_lang(soup)
        nested_dict['available_lang'] = self.get_available_lang(soup)
        nested_dict['comments'] = self.get_comments_count(soup)
        nested_dict['duration'] = self.get_duration(soup)
        nested_dict['duration_sec'] = self.get_duration_sec(soup)
        nested_dict['topic_tags'] = self.get_topic_tags(soup)
        nested_dict['related_talks'] = self.get_related_talks(soup)
        nested_dict['talk_url'] = self.get_talk_url(soup)
        nested_dict['talk_description'] = self.get_talk_description(soup)
        # add transcript if param is set to False (default)
        if not self.exclude:
            nested_dict['transcript'] = self.get_transcript(soup)
        return nested_dict

    def get_data(self):
        """Returns nested dictionary of features from each talk's transcript page."""
        print("Fetching all urls...")
        # define url attribute
        urls = self.remove_urls_with_issues()
        print(f"Scraping {len(urls)} TED talks in '{self.language}'...")
        print(f"Estimated time to complete is {round((.9*len(urls)/60), 1)} minutes\n")
        # iterate through each ted talk transcript url
        for url in urls:
            # make soup
            soup = self.make_soup(url)
            # taste soup
            taster = soup.title.text
            bad_soup = re.search(r'404: Not Found', taster)
            if bad_soup:
                print(f"\nBad soup! TED might not have this talk available in "
                      f"'{self.lang_code}'. Check the url\n{url}\n")
                self.failed_counter += 1
                continue
            # delay between searches
            self.sleep_short()
            # try up to three attempts to scrape data
            for attempt in range(1, 3+1):
                try:
                    # create nested dict
                    self.ted_dict[self.dict_id] = {}
                    # scrape features and add to a nested dict
                    self.scrape_all_features(soup)
                    # indicate successful scrape
                    print(self.dict_id, url)
                    # add 1 to create a new nested dict
                    self.dict_id += 1
                except Exception as e:
                    # if the last attempt fails, update the failed counter
                    # and print the exception & talk url
                    if attempt == 3:
                        self.failed_counter += 1
                        print(f'position: {self.dict_id}, exception: {e}, url: {url}\n')
                        continue
                    # delay before another attempt
                    self.sleep_long()
                # break if no exceptions are raised
                else:
                    break
        print(f"""\nTed.com scraping results:
            \n\t• Successful: {self.dict_id}
            \n\t• Failed: {self.failed_counter}\n""")
        return self.ted_dict
    
    def to_dataframe(self, ted_dict):
        """Creates DataFrame object from dict."""
        df = pd.DataFrame.from_dict(ted_dict, orient='index')
        return df


# Testing

In [52]:
urls = [
    # should be removed
    'https://www.ted.com/talks/zohreh_davoudi_are_we_living_in_a_simulation/',
    'https://www.ted.com/talks/marcus_bullock_an_app_that_helps_incarcerated_people_stay_connected_to_their_families/transcript?language=en', # only in 'en'
    'https://www.ted.com/talks/anton_garcia_abril_how_prefab_homes_can_transform_affordable_housing/',
    
    # good ones
    'https://www.ted.com/talks/jorge_drexler_poetry_music_and_identity/transcript?language=en',
    'https://www.ted.com/talks/sarah_kaminsky_my_father_the_forger/',
]

In [53]:
scraper = TEDscraper(urls=urls, lang_code='en')

In [54]:
scraper.get_data()

Fetching all urls...
Removed the following 3 urls as they have known issues:
 ['https://www.ted.com/talks/zohreh_davoudi_are_we_living_in_a_simulation/transcript?language=en', 'https://www.ted.com/talks/marcus_bullock_an_app_that_helps_incarcerated_people_stay_connected_to_their_families/transcript?language=en', 'https://www.ted.com/talks/anton_garcia_abril_how_prefab_homes_can_transform_affordable_housing/transcript?language=en']

Scraping 2 TED talks in 'English'...
Estimated time to complete is 0.0 minutes

0 https://www.ted.com/talks/jorge_drexler_poetry_music_and_identity/transcript?language=en
1 https://www.ted.com/talks/sarah_kaminsky_my_father_the_forger/transcript?language=en

Ted.com scraping results:
            
	• Successful: 2
            
	• Failed: 0



{0: {'talk_id': '2777',
  'title': 'Poetry, music and identity',
  'speakers': {0: 'Jorge Drexler'},
  'occupations': {0: ['musician', 'poet']},
  'about_speakers': {0: 'Jorge Drexler is a musician and the first Uruguayan to win an Oscar. His music plays with genre and influence, combining subtle harmonies and regional styles with electronic effects.'},
  'views': '1466387',
  'recorded_date': '2017-04-24',
  'published_date': '2017-05-10T15:15:23+00:00',
  'event': 'TED2017',
  'native_lang': 'es',
  'available_lang': ['ar',
   'en',
   'es',
   'fr',
   'he',
   'ko',
   'nl',
   'pt-br',
   'ru',
   'sr',
   'th',
   'zh-cn',
   'zh-tw'],
  'comments': '28',
  'duration': '16M40S',
  'duration_sec': '1000',
  'topic_tags': ['composing',
   'creativity',
   'entertainment',
   'guitar',
   'culture',
   'history',
   'humanity',
   'identity',
   'language',
   'live music',
   'music',
   'poetry',
   'performance',
   'South America',
   'world cultures',
   'TED en Español'],
  'r

#### Errors
* have a csv with urls with errors & clean (need both because some urls don't work for a specific language)
    * if urls in errors CSV, continue (skip)
    * else, see if it's in 'cached'
        * if so, make a note that this will be used
        * continue
    * else, scrape
    * merge new talks df with cached ones df

When should the code above run?
* After getting all urls / cleaning all urls (if urls are provided)
How do I check all the urls efficiently?
* check each url in urls_cached

Creating the cached urls file
* Start with blank file (first time)
    * urls, languages
* Add urls (without the lang) from a successful scrape to a df
* Add the lang_
code to the url column list
    * There will be talks that are only available in certain languages
* When a following scrape session is conducted, do the process again.
    * Only add new urls to the file (prepend)
* How to do it?
    * create method (cache_data)
    * Do this after creating a df...


In [ ]:
def add_cache(df, lang_code='fr'):
    """Add scraped urls to CSV."""
    cached_df = pd.read_csv('../data/urls_test_cached.csv')
    cached_urls = cached_df['url'].array
    for url in df['talk_url']:
        if url in cached_urls: # and lang_code in cached_langs:
            cached_langs = cached_df.loc[cached_df['url'] == url, 'lang_code']
            for code in cached_langs:
                if lang_code not in code:
                    # new lang_code
                    cached_langs
                    # add code to lang_code list
                    cached_langs.update(pd.Series(['test'], 
                else:
                    # get the data from dataset
                    pass
        else:
            cached_df.append(url, list(lang_code))


In [ ]:
df.head()

Solution:
When scraping, don't save them as a list, just do a string (en, es, fr, etc.) in which I can then use s.to_list()

In [ ]:
cached_langs = cached_df.loc[cached_df['url'] == url, 'lang_code']
for code in cached_langs:
    if 'en' in code:
        print('pass')

### Support Code

In [ ]:
def make_soup(url):
    """Returns soup object from a URL."""
    # generate random user-agent
    user_agent = {'User-agent': UserAgent().random}
    # request page and make soup
    page = requests.get(url, headers=user_agent)
    soup = BeautifulSoup(page.content, 'lxml')
    return soup

In [ ]:
# all features
scraper.get_talk_id(soup)
scraper.get_title(soup)
scraper.get_speakers(soup)
scraper.get_occupations(soup)
scraper.get_about_speakers(soup)
scraper.get_views(soup)
scraper.get_recorded_date(soup)
scraper.get_published_date(soup)
scraper.get_event(soup)
scraper.get_native_lang(soup)
scraper.get_available_lang(soup)
scraper.get_comments_count(soup)
scraper.get_duration(soup)
scraper.get_duration_sec(soup)
scraper.get_topic_tags(soup)
scraper.get_related_talks(soup)
scraper.get_talk_url(soup)
scraper.get_talk_description(soup)
scraper.get_transcript(soup)

scraper.get_title(soup)

In [ ]:
# Test urls
urls = [
    'https://www.ted.com/talks/sarah_kaminsky_my_father_the_forger/',
    'https://www.ted.com/talks/jorge_drexler_poetry_music_and_identity/transcript',
    'https://www.ted.com/talks/sir_ken_robinson_do_schools_kill_creativity/',
    'https://www.ted.com/talks/paul_mceuen_and_marc_miskin_tiny_robots_with_giant_potential/transcript',
    'https://www.ted.com/talks/antara_raychaudhuri_and_iseult_gillespie_the_legend_of_annapurna_hindu_goddess_of_nourishment/',
    'https://www.ted.com/talks/diana_reiss_peter_gabriel_neil_gershenfeld_and_vint_cerf_the_interspecies_internet_an_idea_in_progress/',
]

In [ ]:
errors = [
    'https://www.ted.com/talks/marcus_bullock_an_app_that_helps_incarcerated_people_stay_connected_to_their_families/transcript?language=en',
    'https://www.ted.com/talks/marilyn_waring_the_unpaid_work_that_gdp_ignores_and_why_it_really_counts/',
    'https://www.ted.com/talks/zohreh_davoudi_are_we_living_in_a_simulation/',
    'https://www.ted.com/talks/maisie_williams_why_talent_carries_you_further_than_fame/',
    'https://www.ted.com/talks/keith_lowe_why_we_need_to_stop_obsessing_over_world_war_ii/',
    'https://www.ted.com/talks/anton_garcia_abril_how_prefab_homes_can_transform_affordable_housing/',
]
errors_sr = pd.Series(errors, name='errors_urls')

In [ ]:
errors_sr.to_csv('../data/urls_errors.csv', index=False)

## Known Issues

#### Talks that weren't scraped
* Talk is in English but has no English transcript
    * https://www.ted.com/talks/marcus_bullock_an_app_that_helps_incarcerated_people_stay_connected_to_their_families/transcript?language=en
* No 'recorded_date'
    * https://www.ted.com/talks/marilyn_waring_the_unpaid_work_that_gdp_ignores_and_why_it_really_counts/
* Speakers info is missing
    * The name is available elsewhere, but not the occupation
    * https://www.ted.com/talks/zohreh_davoudi_are_we_living_in_a_simulation/
    * https://www.ted.com/talks/maisie_williams_why_talent_carries_you_further_than_fame/
    * https://www.ted.com/talks/keith_lowe_why_we_need_to_stop_obsessing_over_world_war_ii/
    * https://www.ted.com/talks/anton_garcia_abril_how_prefab_homes_can_transform_affordable_housing/
